In [91]:
from glob import glob
import numpy as np
import pandas as pd
import pyrotd
import matplotlib.pyplot as plt
import os
from warnings import filterwarnings
filterwarnings("ignore")
from obspy.geodetics import gps2dist_azimuth
def distcal(stla,stlo,evla,evlo):
    return gps2dist_azimuth(stla,stlo,evla,evlo)[0]/1000

In [92]:
fl = glob("ascii_mw/*")

In [97]:
time_step = 1/200
osc_damping = 0.05
period = np.array([0.01,0.02,0.022,0.025,0.029,0.03,0.032,0.035,0.036,0.04,0.042,0.044,0.045,0.046,
               0.048,0.05,0.055,0.06,0.065,0.067,0.07,0.075,0.08,0.085,0.09,0.095,0.1,0.11,0.12,0.13,
               0.133,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.22,0.24,0.25,0.26,0.28,0.29,0.3,0.32,0.34,
               0.35,0.36,0.38,0.4,0.42,0.44,0.45,0.46,0.48,0.5,0.55,0.6,0.65,0.667,0.7,0.75,0.8,0.85,0.9,0.95,
               1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.2,2.4,2.5,2.6,2.8,3,3.2,3.4,3.5,3.6,3.8,4,4.2,4.4,4.6,4.8,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]);
osc_freqs = np.flip(1/period)
col_names1 = ['EQID','STAID','MW','H_km','evlat','evlon','stlat','stlon']
# create dataframe-2 containing the spectral acclereation
col_names2 = [str(a) + " hz" for a in osc_freqs]
col_names_all = col_names1 + col_names2
df_rot_psa = pd.DataFrame(columns = col_names_all)
for i in range(0,len(fl)):
# for i in range(0,10):
    fn = fl[i]
    eqid = fn.split("_")[-2]
    sta = fn.split("_")[-3].split("/")[-1]
    header = pd.read_csv(fn, nrows = 7, delimiter=": ",index_col=0).T
    header = header.reset_index()
    header.columns = ['sta','stalat', 'stalon', 'evelat', 'evelon', 'ml','mw','dep']
    data = pd.read_csv(fn, delim_whitespace = True, skiprows=17, index_col = False, names=["t","V","H1","H2"])
    time = data.t # time
    ug1 = data.H1 # E component
    ug2 = data.H2 # H component
    header = header[['sta','mw','dep','evelat','evelon','stalat','stalon']]
    eq_info = np.stack([header[v_n] for v_n in header], axis=1).reshape(-1)
    psa_ug1 = pyrotd.calc_spec_accels(time_step, ug1, osc_freqs, osc_damping)
    psa_ug2 = pyrotd.calc_spec_accels(time_step, ug2, osc_freqs, osc_damping)
    gm_psa = np.sqrt(0.5*psa_ug2['spec_accel']**2 + 0.5*psa_ug1['spec_accel']**2)
    tempt =  np.concatenate([[eqid],eq_info, gm_psa])
    df_rot_psa.loc[i] = tempt


In [98]:
df_rot_psa.to_csv('Sa_table.csv')

In [99]:
df_rot_psa

,EQID,STAID,MW,H_km,evlat,evlon,stlat,stlon,0.1 hz,0.105263157895 hz,...,25.0 hz,27.7777777778 hz,28.5714285714 hz,31.25 hz,33.3333333333 hz,34.4827586207 hz,40.0 hz,45.4545454545 hz,50.0 hz,100.0 hz
0,20080302182645,SLVB,4.0,30.4,22.256,101.865,21.330,103.965,0.000007,0.000007,...,0.002229,0.002236,0.002063,0.002047,0.001931,0.001899,0.001848,0.001835,0.001764,0.001782
1,20080321144629,BGVB,2.9,17.0,22.594,108.143,21.290,106.228,0.000008,0.000008,...,0.050776,0.054665,0.054240,0.051269,0.044200,0.045584,0.047238,0.035364,0.029678,0.022806
2,20081008022137,DBVB,3.7,10.0,23.495,101.982,21.390,103.018,0.000017,0.000017,...,0.015652,0.016487,0.016287,0.015845,0.015963,0.015831,0.017431,0.018768,0.016049,0.012500
3,20081017213743,BCVB,3.4,10.7,23.271,103.995,21.885,105.772,0.000007,0.000009,...,0.033135,0.031048,0.030102,0.027602,0.023883,0.025344,0.023213,0.022564,0.021334,0.018689
4,20081017213743,VTVB,3.4,10.7,23.271,103.995,22.252,104.899,0.000010,0.000009,...,0.399152,0.349389,0.328997,0.315165,0.297719,0.299722,0.309237,0.221269,0.198007,0.117411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,20200728012643,VN09,4.1,5.3,20.910,104.716,21.931,103.422,0.000084,0.000096,...,0.024241,0.023168,0.023302,0.023269,0.022503,0.022369,0.022081,0.025552,0.030334,0.023987
934,20200728012643,VN12,4.1,5.3,20.910,104.716,21.532,103.241,0.000132,0.000141,...,0.099863,0.167647,0.188040,0.188501,0.149264,0.134268,0.294884,0.370356,0.282786,0.099549
935,20200731223113,VN09,3.4,8.8,20.924,104.709,21.931,103.422,0.000015,0.000016,...,0.015348,0.014499,0.014251,0.013244,0.012732,0.012560,0.012088,0.012085,0.018754,0.011818
936,20200817011309,TGVB,4.2,6.9,20.907,104.749,21.592,103.418,0.000130,0.000139,...,0.070687,0.068128,0.066640,0.061130,0.062427,0.062545,0.056622,0.049582,0.052478,0.048766
